# Create a NNVM model

In [1]:
import nnvm
import nnvm.symbol as sym
x = sym.Variable("x")
y = sym.Variable("y")
z = sym.elemwise_add(x, y)
compute_graph = nnvm.graph.create(z)
print(compute_graph.ir())

Graph(%x, %y) {
  %2 = elemwise_add(%x, %y)
  ret %2
}


# Compile graph

In [2]:
shape = (4,)
deploy_graph, lib, params = nnvm.compiler.build(compute_graph, target="llvm", shape={"x": shape}, dtype="float32")

# Deploy graph in hardware

In [3]:
import tvm
from tvm.contrib import graph_runtime, util
module = graph_runtime.create(deploy_graph, lib, tvm.cpu(0))

# Run model

## Create input

In [5]:
import numpy as np
x_np = np.array([1, 2, 3, 4]).astype("float32")
y_np = np.array([4, 4, 4, 4]).astype("float32")
module.set_input(x=x_np, y=y_np)

## Run model

In [7]:
module.run()
out = module.get_output(0, out=tvm.nd.empty(shape))
print(out.asnumpy())

[5. 6. 7. 8.]


# Save graph/model

Most deep learning models contains two types of inputs: parameters that remains fixed during inference and data input that need to change for each inference task. Provide the ```params={"y": y_np}``` and compile the graph

In [19]:
deploy_graph, lib, params = nnvm.compiler.build(
    compute_graph, target="llvm", shape={"x": shape}, params={"y": y_np})

In [20]:
temp = util.tempdir()
path_lib = temp.relpath("deploy.so")
lib.export_library(path_lib)

In [22]:
with open(temp.relpath("deploy.json"), "w") as fo:
    fo.write(deploy_graph.json())
with open(temp.relpath("deploy.params"), "wb") as fo:
    fo.write(nnvm.compiler.save_param_dict(params))
print(temp.listdir())

['deploy.params', 'deploy.json', 'deploy.so']


# Load saved model

In [25]:
loaded_lib = tvm.module.load(path_lib)

In [26]:
loaded_json = open(temp.relpath("deploy.json")).read()

In [27]:
loaded_params = bytearray(open(temp.relpath("deploy.params"), "rb").read())

In [28]:
module = graph_runtime.create(loaded_json, loaded_lib, tvm.cpu(0))

In [31]:
params = nnvm.compiler.load_param_dict(loaded_params)
module.load_params(loaded_params)

In [32]:
module.run(x=x_np)

In [33]:
out = module.get_output(0, out=tvm.nd.empty(shape))

In [34]:
print(out.asnumpy())

[5. 6. 7. 8.]
